# Matrix Metrics: Network-Based Systemic Risk Scoring

**An Interactive Tutorial**

This notebook demonstrates the Matrix Metrics framework for measuring systemic risk in financial networks, based on Das (2016).

## 📚 What You'll Learn

1. **The Problem**: Why traditional risk metrics fail
2. **The Solution**: Network-based approach
3. **8 Key Metrics**: S, Di, Centrality, Criticality, R, Δij, I, S̄
4. **Scenario Analysis**: Test what-if scenarios

---

**Key Concepts:**
- **Compromise Score (C)**: Individual institution stress (0 = no stress)
- **Adjacency Matrix (E)**: Network connections between institutions  
- **Systemic Risk (S)**: Aggregate risk = individual stress + network structure

## 1. Setup & Imports

In [ ]:
# Core libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# Set visualization defaults
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("✅ All libraries imported successfully!")

## 2. Load Sample Data

We'll use a 5-institution financial network:

In [ ]:
# Load institution data
institutions = pd.read_csv('matrix_metrics_app/data/sample_data/das_2016_institutions.csv')
print("📊 Institution Data:")
print(institutions)

# Load adjacency matrix
adjacency = pd.read_csv('matrix_metrics_app/data/sample_data/das_2016_adjacency.csv', index_col=0)
print("\n🔗 Network Adjacency Matrix:")
print(adjacency)

# Extract key data
institution_names = institutions['Name'].values
compromise_scores = institutions['CompromiseScore'].values
n_institutions = len(institutions)

print(f"\n✅ Loaded {n_institutions} institutions")

## 3. SystemicRiskCalculator Class

**👉 ACTION REQUIRED**: Copy the `SystemicRiskCalculator` class from `matrix_metrics_app/src/core/systemic_risk.py` into the cell below.

The class should include all these methods:
- `calculate_S()` - Systemic Risk Score
- `calculate_risk_decomposition()` - Risk Decomposition  
- `calculate_centrality()` - Centrality
- `calculate_criticality()` - Criticality
- `calculate_fragility()` - Fragility
- `calculate_normalized_score()` - Normalized Score
- `calculate_risk_increment()` - Risk Increment
- `calculate_cross_risk()` - Cross-Risk Matrix

In [ ]:
# TODO: Copy SystemicRiskCalculator class here from:
# matrix_metrics_app/src/core/systemic_risk.py
#
# For now, run this simplified version:

class SystemicRiskCalculator:
    def __init__(self, compromise_vector, adjacency_matrix):
        self.C = np.array(compromise_vector)
        self.E = np.array(adjacency_matrix)
        self.n = len(self.C)
        self._s_score = None
        self._gradient_s = None
    
    @property
    def S(self):
        if self._s_score is None:
            self._s_score = self.calculate_S()
        return self._s_score
    
    def calculate_S(self):
        c_col = self.C.reshape(-1, 1)
        s_squared = c_col.T @ self.E @ c_col
        return np.sqrt(s_squared.item())
    
    def _get_gradient(self):
        if self._gradient_s is None:
            if self.S == 0:
                self._gradient_s = np.zeros(self.n)
            else:
                self._gradient_s = (self.E + self.E.T) @ self.C / (2 * self.S)
        return self._gradient_s
    
    def calculate_risk_decomposition(self):
        gradient_s = self._get_gradient()
        return self.C * gradient_s
    
    def calculate_centrality(self):
        degrees = np.sum(np.abs(self.E), axis=1) + np.sum(np.abs(self.E), axis=0)
        if np.sum(degrees) > 0:
            return degrees / np.sum(degrees)
        return np.zeros(self.n)
    
    def calculate_criticality(self):
        centrality = self.calculate_centrality()
        if np.sum(self.C) > 0:
            normalized_C = self.C / np.sum(self.C)
            return centrality * normalized_C
        return np.zeros(self.n)
    
    def calculate_fragility(self):
        degrees = np.sum(np.abs(self.E), axis=1) + np.sum(np.abs(self.E), axis=0)
        total_degree = np.sum(degrees)
        if total_degree == 0:
            return 0.0
        degree_shares = degrees / total_degree
        return np.sum(degree_shares ** 2)
    
    def calculate_normalized_score(self):
        norm_c = np.linalg.norm(self.C)
        if norm_c == 0:
            return 0.0
        return self.S / norm_c
    
    def calculate_risk_increment(self):
        Di = self.calculate_risk_decomposition()
        with np.errstate(divide='ignore', invalid='ignore'):
            increment = np.where(self.C != 0, Di / self.C, 0)
        return increment
    
    def calculate_cross_risk(self):
        if self.S == 0:
            return np.zeros((self.n, self.n))
        gradient = self._get_gradient()
        term1 = (self.E + self.E.T) / (2 * self.S)
        term2 = np.outer(gradient, gradient) / self.S
        hessian = term1 - term2
        return hessian

print("✅ SystemicRiskCalculator class defined!")

## 4. Calculate All Metrics

In [ ]:
# Initialize calculator
calc = SystemicRiskCalculator(compromise_scores, adjacency.values)

# Calculate all metrics
S = calc.S
Di = calc.calculate_risk_decomposition()
centrality = calc.calculate_centrality()
criticality = calc.calculate_criticality()
R = calc.calculate_fragility()
S_bar = calc.calculate_normalized_score()
I = calc.calculate_risk_increment()
Delta = calc.calculate_cross_risk()

# Create summary
results = pd.DataFrame({
    'Institution': institution_names,
    'Compromise Score': compromise_scores,
    'Risk Decomposition (Di)': Di,
    'Centrality': centrality,
    'Criticality': criticality,
    'Risk Increment (I)': I
})

print("📊 SYSTEMIC RISK METRICS")
print("="*60)
print(f"\n🎯 Systemic Risk Score (S): {S:.4f}")
print(f"🔗 Network Fragility (R): {R:.4f}")
print(f"📈 Normalized Risk Score (S̄): {S_bar:.4f}")
print(f"\n💡 Interpretation: S̄ = {S_bar:.2f}", end="")
if S_bar > 1:
    print(" > 1: Network AMPLIFIES risk")
elif S_bar < 1:
    print(" < 1: Network DAMPENS risk")
else:
    print(" = 1: Network neutral")
print("\n" + "="*60)
print("\nInstitution-Level Metrics:")
print(results.to_string(index=False))

## 5. Visualize Results

In [ ]:
# Risk Decomposition
fig = px.bar(results.sort_values('Risk Decomposition (Di)', ascending=False),
             x='Institution', y='Risk Decomposition (Di)',
             title='Risk Decomposition by Institution',
             color='Risk Decomposition (Di)',
             color_continuous_scale='Reds')
fig.show()

print(f"💡 Sum of all Di = {np.sum(Di):.4f} = S = {S:.4f} ✓")

## 6. Scenario Analysis: What-If

What happens if we double Bank 5's stress?

In [ ]:
# Create scenario
modified_scores = compromise_scores.copy()
modified_scores[4] = modified_scores[4] * 2  # Double Bank 5's stress

# Recalculate
calc_mod = SystemicRiskCalculator(modified_scores, adjacency.values)
S_mod = calc_mod.S
Di_mod = calc_mod.calculate_risk_decomposition()

# Compare
comparison = pd.DataFrame({
    'Institution': institution_names,
    'Original Di': Di,
    'Modified Di': Di_mod,
    'Change': Di_mod - Di
})

print("📊 SCENARIO: Double Bank 5's stress")
print("="*60)
print(f"Original S: {S:.4f}")
print(f"Modified S: {S_mod:.4f}")
print(f"Change: {S_mod - S:.4f} ({(S_mod-S)/S*100:.1f}%)\n")
print(comparison.to_string(index=False))

# Visualize
fig = px.bar(comparison, x='Institution', y=['Original Di', 'Modified Di'],
             title='Risk Decomposition: Original vs Modified', barmode='group')
fig.show()

## 🎯 Key Takeaways

**What We Learned:**
1. **S = √(C'EC)**: Combines individual stress with network structure
2. **Di (Risk Decomposition)**: Identifies which institutions contribute most (Sum = S)
3. **Centrality vs Criticality**: Position vs position × stress
4. **R (Fragility)**: Higher = more concentrated, vulnerable network
5. **S̄ (Normalized Score)**: > 1 = network amplifies, < 1 = dampens
6. **Scenario Analysis**: Test what-if scenarios easily

**Try It Yourself!**
- Change different institutions' scores
- Remove institutions
- Load your own data

**References:**
- Das, S.R. (2016). Matrix Metrics: Network-Based Systemic Risk Scoring
- GitHub: https://github.com/1sbsharma/SystemicRisk_NetworkMatrix_RiskMetrics